# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [35]:
#Libraries
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [5]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [16]:
space_clean = spaceship.dropna()
space_clean = pd.get_dummies(space_clean, columns=['HomePlanet', 'Destination', 'Cabin'], drop_first=True)
space_clean

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,Cabin_G/992/P,Cabin_G/992/S,Cabin_G/993/S,Cabin_G/994/S,Cabin_G/996/S,Cabin_G/998/S,Cabin_G/999/P,Cabin_G/999/S,Cabin_T/1/P,Cabin_T/3/P
0,0001_01,False,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,...,False,False,False,False,False,False,False,False,False,False
1,0002_01,False,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,False,False,False,False,False,False,False,False,False,False
2,0003_01,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,...,False,False,False,False,False,False,False,False,False,False
3,0003_02,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,...,False,False,False,False,False,False,False,False,False,False
4,0004_01,False,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,False,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,...,False,False,False,False,False,False,False,False,False,False
8689,9278_01,True,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,...,False,False,False,False,False,False,False,False,False,False
8690,9279_01,False,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,...,False,False,False,False,False,False,False,False,False,False
8691,9280_01,False,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,...,False,False,False,False,False,False,False,False,False,False


In [18]:
space_clean.drop(['PassengerId', 'Name'], axis=1, inplace=True)

In [39]:
scaler = StandardScaler()
scaled_columns = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
space_clean[scaled_columns] = scaler.fit_transform(space_clean[scaled_columns])

In [49]:
X = space_clean.drop('Transported', axis=1)
y = space_clean['Transported']

**Perform Train Test Split**

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [53]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20), n_estimators=100, max_samples = 1000)
bagging_reg.fit(X_train, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

- Random Forests

In [59]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [57]:
print(f"Random Forest R2 Score is: {r2_score(y_test, y_pred_rf):.2f}")

Random Forest R2 Score is: 0.37


- Gradient Boosting

In [61]:
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

In [63]:
print(f"Gradient Boosting R2 Score is: {r2_score(y_test, y_pred_gb):.2f}")

Gradient Boosting R2 Score is: 0.44


- Adaptive Boosting

In [65]:
ada_model = AdaBoostRegressor(n_estimators=100, random_state=42)
ada_model.fit(X_train, y_train)
y_pred_ada = ada_model.predict(X_test)

In [67]:
print(f"AdaBoost R2 Score is: {r2_score(y_test, y_pred_ada):.2f}")

AdaBoost R2 Score is: 0.34


Which model is the best and why?

For this data set, the model that performed best was gradient booster with an R2 score of 0.44.